This notebook focuses on feature engineering and cleaning the dataset to generate a final dataset for use with the different algorithms. Below is result of our efforts compressed into one notebook.

In [1]:
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import time

# Round 1
## Functions

In [2]:
'''
This function is responsible for taking a pandas dataframe of taxi data
and returning the cleaned up table necessary for performing analysis.
'''
def clean_data(data):
    # Features not needed for anlaysis
    data.drop('vendorid', axis=1, inplace=True)
    data.drop('extra', axis=1, inplace=True)
    data.drop('mta_tax', axis=1, inplace=True)
    data.drop('tolls_amount', axis=1, inplace=True)
    data.drop('improvement_surcharge', axis=1, inplace=True)
    data.drop('store_and_fwd_flag', axis=1, inplace=True)
    data.drop('ratecodeid', axis=1, inplace=True)
    if 'trip_type ' in data.columns: # need a space after type because the green dataset is a mess
        # trip_type and ehail_fee only exist in the green dataset files
        data.drop('trip_type ', axis=1, inplace=True)
        data.drop('ehail_fee', axis=1, inplace=True)
    
    # Keep only credit card values, then drop the feature
    data.drop(data[data.payment_type != 1].index, inplace=True)
    data.drop('payment_type', axis=1, inplace=True)
    
    # Add trip time and day of week features
    data['trip_time'] = data.apply(time_calc, axis=1)
    data['day_of_week'] = data.apply(weekday_calc, axis=1)
    data['pickup_hour'] = data.apply(hour_extract, axis=1)
    data['month'] = data.apply(month_extract, axis=1)
    data['year'] = data.apply(year_extract, axis=1)
    data['season'] = data.apply(determine_season, axis=1)
    data['DATE'] = data.apply(weatherDate, axis=1)
    data.drop('tpep_pickup_datetime', axis=1, inplace=True)
    data.drop('tpep_dropoff_datetime', axis=1, inplace=True)
    
    # Only keep trips between two minutes and three hours
    data.drop(data[((data.trip_time < 120) | (data.trip_time > 10800))].index, inplace=True)
    
    # Drop total amounts less than zero
    data.drop(data[data.total_amount <= 0].index, inplace=True)

    # Drop fare amounts less than $2.50 (NYC minimum fare)
    data.drop(data[data.fare_amount <= 2.50].index, inplace=True)

    # Calculate tips then drop tip amount and total amount features
    data['tip_percentage'] = data.apply(tip_calc, axis=1)
    data.drop('tip_amount', axis=1, inplace=True)
    data.drop('total_amount', axis=1, inplace=True)
    
    # Drop trips with a distance of 0 or less.
    data.drop(data[data.trip_distance <= 0].index, inplace=True)
    
    # Drop any trips that do not occur within NYC
    data.drop(data[((data.pickup_latitude < 40.459518) | (data.pickup_latitude > 41.175342))].index, inplace=True)
    data.drop(data[((data.dropoff_latitude < 40.459518) | (data.dropoff_latitude > 41.175342))].index, inplace=True)
    data.drop(data[((data.pickup_longitude < -74.361107) | (data.pickup_longitude > -71.903083))].index, inplace=True)
    data.drop(data[((data.dropoff_longitude < -74.361107) | (data.dropoff_longitude > -71.903083))].index, inplace=True)
    
    return data

####################################
### Functions Used by Clean Data ###
####################################

'''
Returns the total number of seconds that a trip took
'''
def time_calc(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    time1 = row['tpep_pickup_datetime']
    time2 = row['tpep_dropoff_datetime']
    return (datetime.datetime.strptime(time2, date_format) - datetime.datetime.strptime(time1, date_format)).total_seconds()

'''
Returns an integer value based on day of the week 
Monday (0) to Sunday (6)
'''
def weekday_calc(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    time1 = datetime.datetime.strptime(row['tpep_pickup_datetime'], date_format)
    return time1.weekday()

'''
Return the hour that pickup occured at
'''
def hour_extract(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    time1 = datetime.datetime.strptime(row['tpep_pickup_datetime'], date_format)
    return time1.hour

'''
Return the month that pickup occured
'''
def month_extract(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    time1 = datetime.datetime.strptime(row['tpep_pickup_datetime'], date_format)
    return time1.month

'''
Return the year that pickup occured
'''
def year_extract(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    time1 = datetime.datetime.strptime(row['tpep_pickup_datetime'], date_format)
    return time1.year

'''
Returns a numerical value based upon the season
winter 1 - 3  0
spring 4 - 6  1
summer 7 - 9  2
fall 10 - 12  3
'''
def determine_season(row):
    
    month = row.month
    
    if month >= 1 and month <= 3:
        return 0
    elif month >= 4 and month <= 6:
        return 1
    elif month >= 7 and month <= 9:
        return 2
    elif month >= 10 and month <= 12:
        return 3
    else:
        return 4

''' 
Used for joining weather data
'''
def weatherDate(row):
    weather_date_format = '%Y%m%d'
    taxi_date_format = '%Y-%m-%d %H:%M:%S'
    time1 = datetime.datetime.strptime(row['tpep_pickup_datetime'], taxi_date_format)
    return int(time1.strftime(weather_date_format))

'''
Calculates the tip percentage and returns the category
'''
def tip_calc(row):
    tip = row['tip_amount']
    total = row['total_amount']
    pct = tip / (total - tip) * 100
    
    # split the pct into categories, this may be too narrow
    if pct <= 5:
        return 0 # 0-5%
    elif pct <= 10:
        return 1 # 5-10%
    elif pct <= 15:
        return 2 # 10-15%
    elif pct <= 20:
        return 3 # 15-20%
    elif pct <= 25:
        return 4 # 20-25%
    elif pct <= 30:
        return 5 # 25-30%
    elif pct <= 35:
        return 6 # 30-35%
    elif pct <= 100: # set this as the max as anything above is likely an outlier. small fares tend to have larger percentage
        return 7 # 35-100%
    else: 
        return 8 # >100% could drop these if we want

## Ouput 1
Might be worthwhile to add multiprocessing, but I'm not too experienced at this so could require some tinkering. Still looking for a good tutorial.

In [3]:
data_directory = 'dataset/taxi/months/'
output_file = 'dataset/clean/cleaned_data.csv'

weather = pd.read_csv('dataset/weather/noaa_central_park_ju15_ju16.csv')

print 'Starting cleaning process at ' + str(time.strftime('%X %x %Z'))
print ''

for f in os.listdir(data_directory):
    if '.csv' in f:
        reader = pd.read_csv(data_directory+f, chunksize=100000)

        t0 = time.time()
        print 'CLEANING DATA: ' + f

        i = 1
        for chunk in reader:
            # convert all column names to lowercase
            chunk.columns = map(str.lower, chunk.columns)

            cd = clean_data(chunk)

            # join weather data and drop the no longer needed DATE field
            combined = pd.merge(left=cd, right=weather, how='left', left_on='DATE', right_on='DATE')
            combined.drop('DATE', axis=1, inplace=True)

            if not os.path.isfile(output_file):
                combined.to_csv(output_file, header='column_names', index=False, columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'trip_distance', 'fare_amount', 'ehail_fee', 'trip_type ', 'trip_time', 'day_of_week', 'pickup_hour', 'month', 'year', 'season', 'tip_percentage', 'PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN'])
            else:
                combined.to_csv(output_file, mode = 'a', header=False,  index=False, columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'trip_distance', 'fare_amount', 'ehail_fee', 'trip_type ', 'trip_time', 'day_of_week', 'pickup_hour', 'month', 'year', 'season', 'tip_percentage', 'PRCP', 'SNWD', 'SNOW', 'TMAX', 'TMIN'])

            print 'STATUS: Chunk ' + str(i) + ' done'
            i += 1

        print  'COMPLETE: ' + f + ' in ' + str(time.time() - t0) + ' seconds'

Starting cleaning process at 18:14:34 11/08/16 EST

CLEANING DATA: green_tripdata_2015-07.csv
STATUS: Chunk 1 done
STATUS: Chunk 2 done
STATUS: Chunk 3 done
STATUS: Chunk 4 done
STATUS: Chunk 5 done
STATUS: Chunk 6 done
STATUS: Chunk 7 done
STATUS: Chunk 8 done
STATUS: Chunk 9 done
STATUS: Chunk 10 done
STATUS: Chunk 11 done
STATUS: Chunk 12 done
STATUS: Chunk 13 done
STATUS: Chunk 14 done
STATUS: Chunk 15 done
STATUS: Chunk 16 done
COMPLETE: green_tripdata_2015-07.csv in 349.912719011 seconds
CLEANING DATA: green_tripdata_2015-08.csv
STATUS: Chunk 1 done
STATUS: Chunk 2 done
STATUS: Chunk 3 done
STATUS: Chunk 4 done
STATUS: Chunk 5 done
STATUS: Chunk 6 done
STATUS: Chunk 7 done
STATUS: Chunk 8 done
STATUS: Chunk 9 done
STATUS: Chunk 10 done
STATUS: Chunk 11 done
STATUS: Chunk 12 done
STATUS: Chunk 13 done
STATUS: Chunk 14 done
STATUS: Chunk 15 done
STATUS: Chunk 16 done
COMPLETE: green_tripdata_2015-08.csv in 326.808700085 seconds
CLEANING DATA: green_tripdata_2015-09.csv
STATUS: Chun

# Round 2
## Functions

In [ ]:
def precision1(row):
    val = float(row.pickup_longitude)
    return round(val, 3)

def precision2(row):
    val = float(row.pickup_latitude)
    return round(val, 3)

def precision3(row):
    val = float(row.dropoff_longitude)
    return round(val, 3)

def precision4(row):
    val = float(row.dropoff_latitude)
    return round(val, 3)

def classify_year(row):
    # 0 is 2015
    # 1 is 2016
    return int(row.year != 2015)

def classify_rain(row):
    return int(row.PRCP > 0)

def classify_snow(row):
    return int(row.SNOW > 0)

def classify_snowdepth(row):
    return int(row.SNWD > 0)

def classify_triptime(row):
    # return minutes
    return int(round(row.trip_time/60))

def clean(data):
    data.drop('trip_type ', axis=1, inplace=True) # drop failed in first run
    data.drop('ehail_fee', axis=1, inplace=True) # drop failed in first run
    data.pickup_longitude = data.apply(precision1, axis=1)
    data.pickup_latitude = data.apply(precision2, axis=1)
    data.dropoff_longitude = data.apply(precision3, axis=1)
    data.dropoff_latitude = data.apply(precision4, axis=1)
    data.year = data.apply(classify_year, axis=1)
    data.PRCP = data.apply(classify_rain, axis=1)
    data.SNOW = data.apply(classify_snow, axis=1)
    data.SNWD = data.apply(classify_snowdepth, axis=1)
    data.trip_time = data.apply(classify_triptime, axis=1)
    return data

In [ ]:
output_file = 'dataset/clean/cleaned_data2.csv'

reader = pd.read_csv('dataset/clean/cleaned_data.csv', chunksize=100000)

for chunk in reader:
    cd = clean(chunk)
    
    if not os.path.isfile(output_file):
        cd.to_csv(output_file, header='column_names', index=False) 
    else:
        cd.to_csv(output_file, mode = 'a', header=False,  index=False)

# Round 3
## Functions

In [ ]:
# Update the min/max coords to reflect our cleaned dataset
min_x_coord = -74.362
min_y_coord = 40.459

max_x_coord = -71.904
max_y_coord = 41.176

step = 0.001  # roughly 111 meters (https://en.wikipedia.org/wiki/Decimal_degrees)

max_matrix_x = int((max_x_coord - min_x_coord) / step)

num_columns = max_matrix_x + 1

def get_xy_matrix(coor, min_coor):
    return int((coor - min_coor) / step)

def get_matrix(matrix_x, matrix_y):
    return (matrix_y * num_columns) + matrix_x

def calc_matrix(row, loc):
    if loc == 'pickup':
        pickup_x = get_xy_matrix(row.pickup_longitude, min_x_coord)
        pickup_y = get_xy_matrix(row.pickup_latitude, min_y_coord)
        pickup_matrix = get_matrix(pickup_x, pickup_y)
        return pickup_matrix
    
    if loc == 'dropoff':
        drop_x = get_xy_matrix(row.dropoff_longitude, min_x_coord)
        drop_y = get_xy_matrix(row.dropoff_latitude, min_y_coord)
        dropoff_matrix = get_matrix(drop_x, drop_y)
        return dropoff_matrix

In [ ]:
output_file = 'dataset/clean/cleaned_data3.csv'

reader = pd.read_csv('dataset/clean/cleaned_data2.csv', chunksize=500000)

i=1
for chunk in reader:
    # Convert to binary instead of T/F
    chunk.year = chunk.year.astype(int)
    chunk.PRCP = chunk.PRCP.astype(int)
    chunk.SNWD = chunk.SNWD.astype(int)
    chunk.SNOW = chunk.SNOW.astype(int)
    # Convert lat/long to a grid matrix
    chunk['pickup_matrix'] = chunk.apply(calc_matrix, args=('pickup',), axis=1)
    chunk['dropoff_matrix'] = chunk.apply(calc_matrix, args=('dropoff',), axis=1)
    chunk.drop('pickup_latitude', axis=1, inplace=True)
    chunk.drop('pickup_longitude', axis=1, inplace=True)
    chunk.drop('dropoff_latitude', axis=1, inplace=True)
    chunk.drop('dropoff_longitude', aAAxis=1, inplace=True)
    
    if not os.path.isfile(output_file):
        chunk.to_csv(output_file, header='column_names', index=False) 
    else:
        chunk.to_csv(output_file, mode = 'a', header=False,  index=False)
    
    if i%5 == 0:
        print('Records complete: ' + str(i*500000))
    i += 1